In [1]:
# Created By: CDT Jack Summers

# imports

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout #standard layers needed for CNN
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator # to create a generator to point at a directory and augment data on the fly
import keras

In [2]:
# data path
# training and validation data
train_data = '../input/gender-classification-dataset/Training'
# Testing data
test_data = '../input/gender-classification-dataset/Validation'

In [3]:
# training generator/augmentation
# The syntax for the generators was adapted from pg 141 of Deep Learning with Python [2].
train_datagen = ImageDataGenerator(rescale = 1/255,
                                   rotation_range = 25,
                                   width_shift_range = 0.1,
                                   height_shift_range = 0.1,
                                   shear_range = 0.1,
                                   zoom_range = 0.1,
                                   horizontal_flip = True,
                                   fill_mode = 'nearest',
                                   validation_split = .2)

# test data generator 
test_datagen = ImageDataGenerator(rescale = 1/255)

In [4]:
# connect generators to data

# training

train_gen = train_datagen.flow_from_directory(directory = train_data,
                                                    subset='training',
                                                    target_size = (92, 114),
                                                    batch_size = 128,
                                                    class_mode = 'binary')
#validating

val_gen = train_datagen.flow_from_directory(directory = train_data,  
                                            batch_size=128, 
                                            subset='validation',
                                            target_size = (92, 114), 
                                            class_mode = 'binary')
                                            
#testing

test_gen = test_datagen.flow_from_directory(directory= test_data,
                                            batch_size=128, 
                                            target_size = (92, 114),
                                            shuffle = False,
                                            class_mode = 'binary')

In [5]:
# callbacks
# The syntax for the callback was adapted from pg 273 of Deep Learning with Python [2].
callback_list = []

es_callback = EarlyStopping(monitor = 'val_acc', 
                           min_delta = .001, # after each epoch we want to see the val accuracy imporve by 0.001
                           patience = 10, # if min_delta not seen after 10 epochs, stop training
                           verbose = 1,
                           restore_best_weights = True) # restore weights of peak val accuracy of that epoch


callback_list.append(es_callback)

In [6]:
# define model
# The syntax for the CNN was adapted from pg 134 of Deep Learning with Python [2].
model = Sequential()
model.add(Conv2D(16, kernel_size = (3, 3), activation = 'relu', input_shape = (92, 114,3))) #kernal size 3x3 standard and so is relu for CNN. input shape is 92x114 image with depth of 3 = color
model.add(MaxPooling2D(2,2))
model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(2, 2))
                                      
model.add(Flatten())

model.add(Dense(units = 64, activation = 'relu')) 
model.add(Dense(units = 1, activation = 'sigmoid')) # 2 options for classifcation. 1 = female 0 = male
model.summary()

In [7]:
# compile model
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['acc'])

In [8]:
# train model

history = model.fit(
    train_gen,
    steps_per_epoch= 294, # 37608 training images / 128 batch size = 293.81
    epochs=1000,
    validation_data=val_gen,
    validation_steps= 74, #9401 images / 128 = 73.44
    callbacks = callback_list)


In [9]:
# plot results
# The syntax for plotting was adapted from pg 74 of Deep Learning with Python [2].
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training Acc.')
plt.plot(epochs, val_acc, 'b', label='Validation Acc.')
plt.title('Training vs. Validation Acc.')

plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training vs Validation Loss')
plt.legend()

plt.show()

In [10]:
# Evalutate with test data. Major Ruiz provided me with the syntax .evaluate(). 
model.evaluate(test_gen)

In [11]:
#save model
#model.save("./history_v3.h5")

In [12]:
#load model
#v1_model = tf.keras.models.load_model('../input/history-v1h5/history_v2.h5')
# v1_model.evaluate(test_gen)